In [4]:
data = "knowledge_graph.projects.json"

In [22]:
#GET PROJECTS FROM FILE
import json

# Provide the correct file path
file_path = r'knowledge_graph.projects.json'

# Try with different encoding
with open(file_path, 'r', encoding='latin-1') as file:
    data_list = json.load(file)
# Extract desired information for each element
projects = []
for data in data_list:
    twitter_link = data.get("links", {}).get("twitter", "")
    category= data.get("category", "")
    if twitter_link and category:
        extracted_info = {
            "id": data["_id"],
            "name": data.get("name", ""),
            "link": twitter_link,
            "category": data.get("category", ""),
            "description": data.get("description", "")
        }
        projects.append(extracted_info)

# Save the extracted information to a new JSON file
output_file_path = "project_list.json"
with open(output_file_path, 'w') as output_file:
    json.dump(projects, output_file, indent=2)

In [ ]:
#GET FOLLOWERS FROM PROJECTS
import asyncio
from twscrape import API, gather
from twscrape.logger import set_log_level
import json

async def main():
    result_list = []

    api = API()  # or API("path-to.db") - default is `accounts.db`
    
    # ADD ACCOUNTS (for CLI usage see BELOW)
    # await api.pool.add_account("ndycuong1", "123456coi", "nguyenduycuong1501@gmail.com", "15011990Ndyc")

    file_path = r'project_list_copy.json'
    with open(file_path, 'r', encoding='latin-1') as file:
        data_list = json.load(file)

    for datalist in data_list:
        data = {'username': {}, 'followers': []}  # Move inside the loop to create a new data dictionary for each user
        twitter_link = datalist.get("link", "")
        username = twitter_link.split("/")[-1]
        data['username'] = username
        data['link'] = twitter_link
        if username:
            try:
                user_by_name = await api.user_by_login(username)
                id = user_by_name.id
                followerses = await gather(api.followers(id, limit=15))
                for follower_data in followerses[1:20]:
                    data['followers'].append({'id': follower_data.id, 'username': follower_data.username})
                filename = "project_followers.json"
                with open(filename, 'a', encoding='utf-8') as jsonfile:
                    json.dump(data, jsonfile, ensure_ascii=False, indent=4)
            except Exception as e:
                print(f"Error fetching data for {username}: {e}")
            
        

        #result_list.append(data)

    # change log level, default info
    set_log_level("DEBUG")

    # Save data to JSON file
    

    print(f"Data saved to {filename}")

if __name__ == "__main__":
    await (main())


In [ ]:
######PYTHON 3.9 - JUPYTER NOTEBOOK
#crawl data profile X from user url

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import json
import re

PATH = r'F:\slide\chromedriver-win64\chromedriver.exe'
#user
#password

# Read the JSON file
with open("project_list.json", 'r', encoding='latin-1') as file:
    project_list = json.load(file)

# Extracting username of projects


urls=[]
# Assuming project_list is a list of projects
for project in project_list:

    # Extract the desired value (Twitter handle) for each project
    urls.append(project["link"])

# Initialize the Chrome driver
driver = webdriver.Chrome(PATH)

# Login to Twitter
driver.get("https://twitter.com/login")
time.sleep(15)

result_list = []

for target_url in urls:
    #url_parts = target_url.split("/")

# Get the last part of the URL
    #username = url_parts[-1]
    o = {}
    twitte=[]
    # Go to the profile page    
    driver.get(target_url)
    time.sleep(10)
    resp = driver.page_source
    
    soup = BeautifulSoup(resp, 'html.parser')
    o['link']=target_url

# Define a regex pattern to extract name and username
    
#get intro
    try:
        profile_header = soup.find("div", {"data-testid": "UserProfileHeader_Items"})
    except:
        None
    try:
        o["profile_joining_date"] = profile_header.find("span", {"data-testid": "UserJoinDate"}).text
    except:
        o["profile_joining_date"] = None

    try:
        o["is_verifed"] = bool(soup.find("svg", {"data-testid": "icon-verified"}))
    except:
        o["is_verifed"] = False


    # Extract location information
    try:
        #location_section = soup.find("span", {"class": "css-901oao css-16my406 r-1bwzh9t r-4qtqp9 r-poiln3 r-1b7u577 r-bcqeeo r-qvutc0"})[1].find("span",{"class":"css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"}).text
        o["profile_location"] = soup.find("span", {"data-testid":"UserLocation"}).text
            #     location_parts = location_section.find_all("span", recursive=False)
    #     o["profile_location"] = location_parts[0].text.strip()
    except:
        o["profile_location"] = None
#get followers       
    follower_url=target_url+'/followers'
    driver.get(follower_url)
    time.sleep(5)
    scroll_down_count = 3  # Change this to the number of times you want to scroll down
    for _ in range(scroll_down_count):
    # Scroll down to the bottom of the page
        #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        #driver.execute_script("window.scrollBy(0, window.innerHeight);")
        scroll_distance = 2000
        driver.execute_script(f"window.scrollBy(0, {scroll_distance});")

        # Wait for a short time to let the page load more content
        time.sleep(10)

        resp = driver.page_source
        soup = BeautifulSoup(resp, 'html.parser')
    
        block = soup.find_all("div", {"data-testid":"cellInnerDiv"})
        followers=[]
    
    
        for user in block:
            try:
                #username = user.find("div", {"class": "css-1dbjc4n r-1adg3ll r-1ny4l3l"}).find("a", {"class": "css-4rbku5 css-18t94o4 css-1dbjc4n r-1loqt21 r-1wbh5a2 r-dnmrzs r-1ny4l3l"}).text
                userurl = user.find("div", {"class": "css-1dbjc4n r-1adg3ll r-1ny4l3l"}).find("div", {"class": "css-901oao css-1hf3ou5 r-1bwzh9t r-18u37iz r-37j5jr r-1wvb978 r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-qvutc0"}).text
                followers.append( userurl)
            except:
                pass       
    o['followers']=followers

    result_list.append(o)

# Save data to JSON file
    filename = "project_followers.json"
    with open(filename, 'w', encoding='utf-8') as jsonfile:
        json.dump(result_list, jsonfile, ensure_ascii=False, indent=4)

    print(f"Data saved to {filename}")
driver.quit()

In [3]:
# FIX FILE
import json

with open('profile_and_tweets_followers.json', 'r',encoding='utf-8') as file:
    data = file.read()

# Replace '}{' with '},{'
corrected_data = data.replace('}{', '},{')

with open('fixed_profile_and_tweets_followers.json', 'w',encoding='utf-8') as file:
    file.write(corrected_data)

In [ ]:
import asyncio
from twscrape import API, gather
from twscrape.logger import set_log_level
import json
from datetime import datetime

class DateTimeEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, datetime):
            return o.isoformat()
        elif hasattr(o, '__dict__'):
            return o.__dict__
        return super().default(o)

async def get_user_additional_info(api, user_id):
    try:
        user = await api.user_by_id(user_id)

        if user is not None:
            user_details = {
                'id': user.id,
                'username': user.username,
                'displayName': user.displayname,
                'url': user.url,
                'intro': user.rawDescription,
                'join_date': user.created,
                'location': user.location,
                'friends_count': user.friendsCount,
                'favorite_count':user.favouritesCount,
                'followers_count': user.followersCount,
                'media_count': user.mediaCount,
                'status_count': user.statusesCount,
                'listed_count': user.listedCount,
                'is_protected': user.protected,
                'is_verified': user.verified,
                'ref': user. descriptionLinks,
            }
            return user_details
        else:
            return None

    except Exception as e:
        print(f"Error fetching user details: {e}")
        return None

async def crawl_user_profile_and_tweets(api, username, limit=20):
    try:
        # Get user information
        user = await api.user_by_login(username)

        if user is not None and hasattr(user, 'id'):
            # Get user tweets
            tweets = await gather(api.user_tweets(user.id, limit=limit))

            # Create a list of dictionaries with user, tweet, and additional user information
            data = {'user': {}, 'tweets': []}
            user_additional_info = await get_user_additional_info(api, user.id)
            if user_additional_info:
                data['user'] = {
                'id': user.id,
                'username': user.username,
                'displayName': user.displayname,
                'url': user.url,
                'intro': user.rawDescription,
                'join_date': user.created,
                'location': user.location,
                'friends_count': user.friendsCount,
                'favorite_count':user.favouritesCount,
                'followers_count': user.followersCount,
                'media_count': user.mediaCount,
                'status_count': user.statusesCount,
                'listed_count': user.listedCount,
                'is_protected': user.protected,
                'is_verified': user.verified,
                'ref': user. descriptionLinks,
                }

            for tweet in tweets:
                tweet_details = await api.tweet_details(tweet.id)
                data['tweets'].append({
                    'user':tweet.user.username,
                    'tweet_id': tweet.id,
                    'tweet_content': tweet.rawContent,
                    'tweet_created_at': tweet.date,
                    'like_count': tweet.likeCount,
                    'retweet_count': tweet.retweetCount,
                    'comment_count': tweet_details.replyCount,
                    'view_count': tweet.viewCount ,
                    'quote_count': tweet.quoteCount,
                    'hashtags': tweet.hashtags,
                    'place': tweet.place,
                    'mentioned_users': tweet.mentionedUsers
                })
            
            # Export data to a JSON file
            filename = r"profile_and_tweets_followers.json"
            with open(filename, 'a', encoding='utf-8') as file:
                json.dump(data, file, ensure_ascii=False, indent=4, cls=DateTimeEncoder)

            print(f"Data for {username} has been successfully exported to file ")
        else:
            print(f"User {username} not found or has no ID.")
    except Exception as e:
        print(f"Error: {e}") 
                 

async def main():
    api = API()
    #await api.pool.add_account("your_username", "your_password", "your_email@example.com", "your_email_password")
    await api.pool.login_all()

    json_file_path = r'part3.json'

    # Read the JSON file
    with open(json_file_path, 'r', encoding='utf-8') as file:
        user_list = json.load(file)
        
        

    for username in user_list:
    # Call the crawl_user_profile_and_tweets function
        await crawl_user_profile_and_tweets(api, username, limit=20)

    set_log_level("DEBUG")

    # Other actions you may want to perform using the API

if __name__ == "__main__":
    await main()

In [6]:
#CHECK DUPLICATE
import json

# Read the JSON file
filename = "project_followers_fixed.json"
with open(filename, 'r') as jsonfile:
    data = json.load(jsonfile)

# Extract unique usernames of followers
follower_usernames = set()
duplicate_usernames = set()
for entry in data:
    followers = entry.get("followers", [])
    for follower in followers:
        username = follower.get("username")
        if username:
            if username in follower_usernames:
                duplicate_usernames.add(username)
            else:
                follower_usernames.add(username)

# Save the unique usernames to a new JSON file
output_filename = "project_follower_usernames.json"
with open(output_filename, 'w') as output_file:
    json.dump(list(follower_usernames), output_file, indent=4)

print(f"Usernames of followers saved to {output_filename}")


Usernames of followers saved to project_follower_usernames.json


In [1]:
import json

# File path of the JSON file
file_path = "Bigdata.json"
file_path1 = "Bigdata1.json"

# Read the contents of the JSON file
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Remove duplicates from the data
data = list(set(json.dumps(item) for item in data))
data = [json.loads(item) for item in data]

# Write the updated data back to the JSON file
with open(file_path1, "w", encoding="utf-8") as file:
    json.dump(data, file, indent=4)

print("Duplicates removed from the JSON file.")

Duplicates removed from the JSON file.


In [2]:
import ijson
import json

def process_user_objects(filename):
    unique_users = {}
    with open(filename, 'r',encoding="utf-8") as file:
        # ijson sẽ trả về mỗi đối tượng trong danh sách
        for user_object in ijson.items(file, 'item'):
            user_id = user_object['user']['id']
            if user_id not in unique_users:
                unique_users[user_id] = user_object

    return list(unique_users.values())

# Xử lý tệp JSON và nhận về danh sách không trùng lặp
unique_user_objects = process_user_objects('Bigdata.json')

# Ghi dữ liệu không trùng lặp vào một tệp mới
with open('output_file.json', 'w',encoding="utf-8") as file:
    json.dump(unique_user_objects, file, indent=4)